In [0]:
dbutils.widgets.text('p_file_date', '2021-03-21')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
spark.sql(f"""
              CREATE TABLE IF NOT EXISTS f1_presentation.calculated_race_results
              (
                  race_year INT,
                  team_name STRING,
                  driver_id INT,
                  driver_name STRING,
                  race_id INT,
                  position INT,
                  points INT,
                  calculated_points INT,
                  created_date TIMESTAMP,
                  updated_date TIMESTAMP
              )
              USING DELTA
""")

In [0]:
spark.sql(f"""
            CREATE OR REPLACE TEMP VIEW race_results_updated
            AS 
            SELECT rc.race_year, 
                co.name AS team_name,
                dr.driver_id,
                dr.name AS driver_name, 
                rc.race_id,
                re.position,
                re.points,
                11 - re.position AS calculated_points 
            FROM f1_processed.results re 
            JOIN f1_processed.drivers dr ON (re.driver_id = dr.driver_id)
            JOIN f1_processed.constructors co ON (re.constructor_id = co.constructor_id)
            JOIN f1_processed.races rc ON (re.race_id = rc.race_id)
            WHERE re.position <= 10
              AND re.file_date = '{v_file_date}'
""")

In [0]:
spark.sql(f"""
MERGE INTO f1_presentation.calculated_race_results tgt
USING race_results_updated upd
ON (tgt.driver_id = upd.driver_id AND tgt.race_id = upd.race_id)
WHEN MATCHED THEN
  UPDATE SET
    tgt.position = upd.position,
    tgt.points = upd.points,
    tgt.calculated_points = upd.calculated_points,
    tgt.updated_date = current_timestamp
WHEN NOT MATCHED
  THEN INSERT (race_year, team_name, driver_id, driver_name, race_id, position, points, calculated_points, created_date)
       VALUES (race_year, team_name, driver_id, driver_name, race_id, position, points, calculated_points, current_timestamp)
""")

In [0]:
%sql select count(1) from race_results_updated;

In [0]:
%sql select count(1) from f1_presentation.calculated_race_results;

Aplicamos dos condiciones: 
- 11 - re.position AS calculated_points : como no se usaba el mismo formato de puntos que se usa actualmente, se asignó que cada piloto que ganara una carrera, por ejemplo anterior, se le restaría su posición. En este caso, Ayrton Senna ganó una carrera y sumo 10 puntos, al aplicar la condición, se queda con un total de 10 puntos y se aplicó la misma condición para los pilotos actuales.
- WHERE re.position <= 10 : solo considera los primeros 10 porque son los pilotos que suman puntos. 